<a href="https://colab.research.google.com/github/Sanazzzmi/Sentiment_analysis_of_the_Persian_text_with_word_embedding_-_frequency_word/blob/main/Sentiment_analysis_of_the_Persian_text_with_word_embedding_%26_frequency_word_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### install and import data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install hazm
!test -f resources-0.5.zip || curl -LO https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip
!test -d resources || ( mkdir -p resources && cd resources && unzip ../resources-0.5.zip )

import hazm
from __future__ import unicode_literals
from hazm import *
import pandas as pd
import random

#dataset
dataset = pd.read_excel('/content/drive/MyDrive/data/digikala_data/digikala.xlsx')


     |████████████████████████████████| 316 kB 5.0 MB/s 
     |████████████████████████████████| 233 kB 26.0 MB/s 
     |████████████████████████████████| 1.4 MB 31.5 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=ebb111bc26a5019d6b814d87ea059721f344b610f99e900de62b4383cd37922c
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154002 sha256=1e580bb1d9dd5f5236a1e4dac68f365a428a0c4f518befa4ef168811b90cfbda
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  U

In [ ]:
dataset.comment = dataset.comment.apply(str)
dataset.head()

,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
0,3692,ماوس بی‌سیم لاجیتک مدل M325,IT,989472,0,0,verified,\N,NaN,واقعا عالیه. من که ازش خیلی راضیم,NaN,NaN
1,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,3862150,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
2,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,626843,1,0,verified,not_recommended,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN
3,120499,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,786887,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,67200,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,854531,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
!gunzip /content/cc.fa.300.bin.gz
!pip install fasttext

import fasttext 

%time
model = fasttext.load_model("/content/cc.fa.300.bin")

--2022-02-01 15:10:07--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4502524724 (4.2G) [application/octet-stream]
Saving to: ‘cc.fa.300.bin.gz’

cc.fa.300.bin.gz    100%[===================>]   4.19G  25.0MB/s    in 1m 57s  

2022-02-01 15:12:04 (36.7 MB/s) - ‘cc.fa.300.bin.gz’ saved [4502524724/4502524724]

     |████████████████████████████████| 68 kB 3.1 MB/s 
  Using cached pybind11-2.9.0-py2.py3-none-any.whl (210 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3124503 sha256=442a2715e9126533aaa3dc209581f0aae828aa0b7d7da2ae3b2b7b0875d60f43
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built 

### train test split for CNN modle

In [ ]:
dataset.groupby('recommend').count()

,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,title,comment,advantages,disadvantages
recommend,,,,,,,,,,,
\N,36382,36382,36382,36382,36382,36382,36382,35106,36382,16241,11953
no_idea,10536,10536,10536,10536,10536,10536,10536,10207,10536,3840,3746
not_recommended,16110,16110,16110,16110,16110,16110,16110,15363,16110,4881,6256
recommended,36972,36972,36972,36972,36972,36972,36972,36054,36972,17279,12166


In [ ]:
dataset.recommend[dataset.recommend == '\\N'] = 2
dataset.recommend[dataset.recommend == 'no_idea'] = 2
dataset.recommend[dataset.recommend == 'not_recommended'] = 3
dataset.recommend[dataset.recommend == 'recommended'] = 1

df_final = dataset[['recommend', 'comment']]
df_final.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

,recommend,comment
0,2,واقعا عالیه. من که ازش خیلی راضیم
1,1,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
2,3,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...
3,2,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...
4,2,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...


In [ ]:
def CleanPersianText(text):
  _normalizer = hazm.Normalizer()
  text = _normalizer.normalize(text)
  return text

revlist = list(map(lambda x: [CleanPersianText(x[0]),x[1]],zip(dataset['comment'],dataset['recommend'])))
pos=list(filter(lambda x: x[1] == 1,revlist))
nat=list(filter(lambda x: x[1] == 2,revlist))
neg=list(filter(lambda x: x[1] == 3,revlist))

revlist_shuffle = pos[:] + neg[:]
random.shuffle(revlist_shuffle)

print("Posetive count {}".format(len(pos)))
print("Negetive count {}".format(len(neg)))
print("Natural  count {}".format(len(nat)))
print()
print("Total    count {}".format(len(revlist)))
print()
print("Posetive count : ","\n",pos[random.randrange(1,len(pos))])
print("Negetive count : ","\n",neg[random.randrange(1,len(neg))])
print("unknown  count : ","\n",nat[random.randrange(1,len(nat))])
print("Total    count {}".format(len(revlist_shuffle)))

Posetive count 36972
Negetive count 16110
Natural  count 46918

Total    count 100000

Posetive count :  
 ['اگه میخواید یه اسپیکر همراتون باشه که همیشه و همه جا ببرید و وصل کنید به موبایل خوبه \nدر نگاه اول تعجب میکنید که اسپیکر به این کوچیکی همچین صدایی داره\nاما اگر میخواید یه اسپیکر عالی داشته باشید این اون عالیه نیست \nمدل Square Box ۲ فوق العاده ست یا سونی SRRXB۱۰۰ این مدل فقط برای این خوبه که توی محیط کار یه آهنگ با صدای کم گوش کنید \nتو صدای بلند حرفی برای گفتن نداره نسبت به دو مدلی هم که گفتم کیفیت و تفکیک صدای متوسطی داره، اما با این حال نسبت به سایر برندهای این دره قیمت بهتره', 1]
Negetive count :  
 ['به گوشی نوت ۵ نمیخورد.', 3]
unknown  count :  
 ['دیروز به\u200cدستم رسید، مونتاژش کار سختی نبود، از لحاظ کیفیت عالیه ولی خوب قیمت بالایی داره، اگه سوالی بود در خدمتم', 2]
Total    count 53082


In [ ]:
vector_size = 300 
max_no_tokens = 20 
import numpy as np
import keras.backend as K
train_size = int(0.9*(len(revlist_shuffle)))
test_size = int(0.1*(len(revlist_shuffle)))#lstm
indexes = set(np.random.choice(len(revlist_shuffle), train_size + test_size, replace=False))

x_train = np.zeros((train_size, max_no_tokens, vector_size), dtype=K.floatx())
y_train = np.zeros((train_size, 2), dtype=np.int32)

x_test = np.zeros((test_size, max_no_tokens, vector_size), dtype=K.floatx())
y_test = np.zeros((test_size, 2), dtype=np.int32)


for i, index in enumerate(indexes):
  text_words = hazm.word_tokenize(revlist_shuffle[index][0])
  for t in range(0,len(text_words)):
    if t >= max_no_tokens:
      break

    if text_words[t] not in model.words:
      continue
    if i < train_size:
      x_train[i, t, :] = model.get_word_vector(text_words[t])
    else:
      x_test[i - train_size, t, :] = model.get_word_vector(text_words[t])

  if i < train_size:
    y_train[i, :] = [1.0, 0.0] if revlist_shuffle[index][1] == 3 else [0.0, 1.0]
  else:
    y_test[i - train_size, :] = [1.0, 0.0] if revlist_shuffle[index][1] == 3 else [0.0, 1.0]
   
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((47773, 20, 300), (5308, 20, 300), (47773, 2), (5308, 2))

### **Article idea**

In [ ]:
text = [x[0] for x in revlist_shuffle ]
y = [x[1] for x in revlist_shuffle ]

In [ ]:
from keras.preprocessing.text import Tokenizer
token = Tokenizer()
token.fit_on_texts(text)

In [ ]:
vocab_size  = len(token.word_index) + 1
vocab_size

51526

In [ ]:
encoded_text = token.texts_to_sequences(text)
print(encoded_text[:3])

[[7, 13, 723, 1427, 78, 44, 4, 55, 27, 403, 1, 8329, 256, 12], [2, 930, 709, 24, 24, 145, 1, 22785, 45], [15, 172, 183, 1, 990, 123, 11, 61, 554, 1, 66, 1904]]


In [ ]:
from keras.preprocessing.sequence import pad_sequences
max_length = 120
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')


In [ ]:
import numpy as np
d = 300
word_vector_matrix = np.zeros((vocab_size, d))


not_word = []
have_word = []

for word, index in token.word_index.items()  :# هر کلمه یه شناسه داره
    vector = model.get_word_vector(word)
    if vector is not None:
        word_vector_matrix[index] = vector
        have_word.append(word)
    else:
        not_word.append(word)    

In [ ]:
yy = []
for x in y:
  if x == 1:
    yy.append(1)
  else:
    yy.append(0)

In [ ]:
#Divide texts by labels one and zero
df_y = pd.DataFrame(np.array(yy),columns=['a'])
df_one = df_y[df_y['a']==1].index.values.astype(int)
df_zero = df_y[df_y['a']==0].index.values.astype(int)

text_one = []
for x in df_one:
    text_one.append(text[x])

text_zero = []
for x in df_zero:
    text_zero.append(text[x])

join_text_zero = [' '.join(text_zero)]
join_text_one = [' '.join(text_one)]

df_join_text_zero = pd.DataFrame(join_text_zero[0:],columns= ["tex"])
df_join_text_one = pd.DataFrame(join_text_one[0:],columns= ["tex"])

df_have_word = pd.DataFrame({'word':have_word})
df_token = pd.DataFrame.from_dict(token.word_index.items())

df_text_one = pd.DataFrame(text_one[0:],columns= ["tex"])
df_text_zero = pd.DataFrame(text_zero[0:],columns= ["tex"])

from collections import Counter
count_zero = df_join_text_zero['tex'].str.split().apply(Counter)
count_one = df_join_text_one['tex'].str.split().apply(Counter)

df_counter_zero = pd.DataFrame.from_dict(count_zero[0], orient='index').reset_index()
df_counter_one = pd.DataFrame.from_dict(count_one[0], orient='index').reset_index()
df_counter_zero.columns = ['word', 'value']
df_counter_one.columns = ['word', 'value']

df_token.columns = ['word', 'id']

sentiment_zero =df_have_word.merge(df_counter_zero, left_on='word', right_on='word')[['value','word']]
sentiment_one = df_have_word.merge(df_counter_one, left_on='word', right_on='word')[['value','word']]

#Words found in both positive and negative sentences
common = sentiment_one.merge(sentiment_zero,on=['word','word'])
common.columns = ['value_one', 'word' , 'value_zero']

common["max"] = common[["value_one", "value_zero"]].max(axis=1)
common["min"] = common[["value_one", "value_zero"]].min(axis=1)


common["lable"] = np.where(common["value_one"] == common["max"], 1, 0)
common["frequent_negative_sentence"] = np.where(common["lable"] == 1 , common["min"], common["max"])
common["frequent_positive_sentence"] = np.where(common["lable"] == 0 , common["min"], common["max"])

common_df_final = common.drop('value_one', 1)
common_df_final = common_df_final.drop('value_zero', 1)
common_df_final = common_df_final.drop('max', 1)
common_df_final = common_df_final.drop('min', 1)

#Those words that are seen in positive sentences and not seen in negative sentences
just_one = sentiment_one[(~sentiment_one.word.isin(common.word))&(~sentiment_one.word.isin(common.word))]
just_one = just_one.assign(lable = 1)
just_one = just_one.assign(frequent_negative_sentence = 0)
just_one.columns = ['frequent_positive_sentence', 'word' , 'lable','frequent_negative_sentence']

#Those words that are seen in negative sentences and not seen in positive sentences
just_zero = sentiment_zero[(~sentiment_zero.word.isin(common.word))&(~sentiment_zero.word.isin(common.word))]
just_zero = just_zero.assign(lable = 0)
just_zero = just_zero.assign(frequent_posoitve_sentence = 0)
just_zero.columns = ['frequent_negative_sentence', 'word' , 'lable','frequent_positive_sentence']

frames = [common_df_final,just_zero, just_one]
result = pd.concat(frames)

In [ ]:
result['WI_positive_word'] = np.where(result['lable']== 1 , 1- result['frequent_negative_sentence']/ result['frequent_positive_sentence'], None)
result['WI_negative_word'] = np.where(result['lable']== 0 , 1- result['frequent_positive_sentence']/ result['frequent_negative_sentence'], None)
result['WI'] = result['WI_positive_word'].fillna(result['WI_negative_word'])

#result.index[result['lable'] == 0].tolist()

In [ ]:
df = pd.DataFrame(list(zip(text, y)), columns =['text', 'lable'])

In [ ]:
word_have_list= list(result.word)
word_have_set = set(word_have_list)
result_word_list = result['word'].tolist()
result_WI_list = result['WI'].tolist()

def word_WI_finder(x):
    df_words = set(x.split(' '))
    extract_words =  word_set.intersection(df_words)
    #index_word = result_word_list.index(extract_words)
    
    
    return (extract_words)


word_set = word_have_set

df['ddd_WI'] = df.text.apply(word_WI_finder)

In [ ]:
WI_ww = []
for x in df['ddd_WI'][:]:
    WI_w = []
    for y in x : 
        ind = result_word_list.index(y)
        WI_w.append(result_WI_list[ind])
    WI_ww.append(WI_w)
    
WI_ww_new = [[] if x==[0] else x for x in WI_ww]
WI_ww_neww = [[] if x==[0,0] else x for x in WI_ww_new]
Weight_sentence = [[float(j)/sum(i) for j in i] for i in WI_ww_neww[:]]



In [ ]:
sentence_ss = []
for i in range(len(df['ddd_WI'][:])):
    #print(df['ddd_WI'][x])
    vv = []
    for j in range(len(df['ddd_WI'][i])):
        #print(list(df['ddd_WI'][i])[j])
        if len(Weight_sentence[i]) > 1:
            v = []
            for num in range(d):
                vec = Weight_sentence[i][j] * float(model.get_word_vector(list(df['ddd_WI'][i])[j])[num])
                v.append(vec)
            #print(v)
            vv.append(v)
        #print(vv)
        else:
            vv.append(model.get_word_vector(list(df['ddd_WI'][i])[j]).astype(np.float))
            
    s = [sum(x) for x in zip(*vv)]
    sentence_ss.append(s)

sentence_ss[0]

data = pd.DataFrame(sentence_ss[0:] , columns = list(range(d)))


In [ ]:
r = [a for a in range(len(data))]
random.shuffle(r)
train_size = int(0.9*(len(r)))
test_size = int(0.1*(len(r)))
train_index = r[:train_size]
test_index = r[train_size:]
x_train = data.iloc[train_index]
x_test = data.iloc[test_index]

li_y_train = [yy[i] for i in train_index]
li_y_test = [yy[i] for i in test_index]
y_train = pd.DataFrame(li_y_train)
y_test = pd.DataFrame(li_y_test)
y_train = pd.DataFrame(li_y_train)
y_test = pd.DataFrame(li_y_test)

### save train & test data

In [ ]:
x_train.to_csv('x_train.csv')
!cp x_train.csv "drive/My Drive/data/digikala_data/"

y_train.to_csv('y_train.csv')
!cp y_train.csv "drive/My Drive/data/digikala_data/"

x_test.to_csv('x_test.csv')
!cp x_test.csv "drive/My Drive/data/digikala_data/"

y_test.to_csv('y_test.csv')
!cp y_test.csv "drive/My Drive/data/digikala_data/"

In [ ]:
pd.read_csv('/content/drive/MyDrive/data/digikala_data/y_train.csv')

,Unnamed: 0,0
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
47768,47768,1
47769,47769,1
47770,47770,1
47771,47771,1


### Preparation data 

In [2]:
import pandas as pd

x_train = pd.read_csv('/content/drive/MyDrive/data/digikala_data/x_train.csv')
x_train = x_train.drop('Unnamed: 0', 1)

y_train = pd.read_csv('/content/drive/MyDrive/data/digikala_data/y_train.csv')
y_train = y_train.drop('Unnamed: 0', 1)

x_test = pd.read_csv('/content/drive/MyDrive/data/digikala_data/x_test.csv')
x_test = x_test.drop('Unnamed: 0', 1)

y_test = pd.read_csv('/content/drive/MyDrive/data/digikala_data/y_test.csv')
y_test = y_test.drop('Unnamed: 0', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]


In [3]:
x_train = x_train.fillna(0)
y_train = y_train.fillna(0)
x_test = x_test.fillna(0)
y_test = y_test.fillna(0)


### CNN

In [ ]:
from keras import optimizers
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Embedding, Dropout
from keras.layers import GlobalMaxPool1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import CuDNNLSTM, LSTM, Bidirectional
from keras.layers.convolutional import Conv1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional,Conv2D
from keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard

In [ ]:
#https://stackoverflow.com/questions/62475807/cnn-on-tfidf-as-input
# create model
inp = Input(shape=(1,300))
conv2 = Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(inp)
drop21 = Dropout(0.2)(conv2)

conv22 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(drop21)
drop22 = Dropout(0.2)(conv22)
conv23 = Conv1D(filters=32, kernel_size=5, activation='relu', padding='same')(drop22)
drop23 = Dropout(0.2)(conv23)
conv24 = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(drop23)
drop24 = Dropout(0.2)(conv24)

pool2 = Flatten()(drop24) # this is an option to pass from 3d to 2d
out = Dense(2, activation='softmax')(pool2) # the output dim must be equal to the num of class if u use softmax

model = Model(inp, out)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(np.expand_dims(np.array(x_train), axis=1), np.array(y_train).flatten(), epochs=200)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 300)]          0         
                                                                 
 conv1d (Conv1D)             (None, 1, 128)            192128    
                                                                 
 dropout (Dropout)           (None, 1, 128)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 64)             41024     
                                                                 
 dropout_1 (Dropout)         (None, 1, 64)             0         
                                                                 
 conv1d_2 (Conv1D)           (None, 1, 32)             10272     
                                                                 
 dropout_2 (Dropout)         (None, 1, 32)             0     

In [ ]:
model.evaluate(np.expand_dims(np.array(x_test), axis=1), np.array(y_test).flatten(), batch_size=32, verbose=1)

166/166 [==============================] - 1s 2ms/step - loss: 0.6579 - accuracy: 0.8623


[0.6579249501228333, 0.8623092770576477]

### **classification**

### xgboost

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(x_train , label = y_train)
dtest = xgb.DMatrix(x_test , label = y_test)
param = {
    'max_depth': 10,  # the maximum depth of each tree
    'eta': 0.1,  # the training step for each iteration
    #'silent': 1,  # logging mode - quiet
    'objective': 'binary:logistic',  
    'eval_metric': 'error',
    #'num_class': 2}  # the number of classes that exist in this datset
}

num_round = 500  # the number of training iterations
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)
import numpy as np
best_preds = np.asarray([np.argmax(line) for line in preds])
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, best_preds , normalize=True)*100)

30.646072706724432


### LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0,solver='lbfgs', max_iter=100 ).fit(x_train, y_train.values.ravel())
y_pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.8527029572424185

### MLPClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(7, 2), random_state=1)
clf.fit(x_train, y_train.values.ravel())
y_pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6935392729327557

### DecisionTree

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7291391975889998

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski')
classifier.fit(x_train, y_train.values.ravel())
y_pred = classifier.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7894142022979845

### SVM

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(x_train, y_train.values.ravel())
y_pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8730457713317009